# Task 1 Part 2: Customer Segmentation using Support Vector Machines (SVM)

## Objective
Develop a machine learning model using SVM techniques to classify customers as "High value" or "Low value" based on features like Age, Income, Spending Score, and Region.

## Learning Outcomes
- Explain the concepts and mathematics behind SVMs and their applications
- Describe the structure and function of neural networks, including their learning processes
- Identify and evaluate the fundamental concepts of deep learning, including popular architectures and their applications

---
## 1. Import Libraries and Load Data

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay
)
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('../Task_1_Part_2_Assets/customersegmentation.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nFirst 10 rows of the dataset:")
df.head(10)

---
## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Basic information about the dataset
print("Dataset Info:")
print(df.info())
print("\n" + "="*50)
print("\nStatistical Summary:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

# Check class distribution
print("\nTarget Variable Distribution (Customer_Category):")
print(df['Customer_Category'].value_counts())
print(f"\nClass 0 (Low Value): {(df['Customer_Category'] == 0).sum()} customers")
print(f"Class 1 (High Value): {(df['Customer_Category'] == 1).sum()} customers")

In [ ]:
# Visualize the target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Target distribution
ax1 = axes[0]
target_counts = df['Customer_Category'].value_counts()
colors = ['#ff6b6b', '#4ecdc4']
ax1.bar(['Low Value (0)', 'High Value (1)'], target_counts.values, color=colors, edgecolor='black')
ax1.set_title('Customer Category Distribution', fontsize=14, fontweight='bold')
ax1.set_ylabel('Count')
for i, v in enumerate(target_counts.values):
    ax1.text(i, v + 10, str(v), ha='center', fontweight='bold')

# Region distribution
ax2 = axes[1]
region_counts = df['Region'].value_counts()
ax2.bar(region_counts.index, region_counts.values, color='steelblue', edgecolor='black')
ax2.set_title('Region Distribution', fontsize=14, fontweight='bold')
ax2.set_ylabel('Count')
for i, v in enumerate(region_counts.values):
    ax2.text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize numerical feature distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

numerical_features = ['Age', 'Income', 'Spending_Score']

for i, feature in enumerate(numerical_features):
    ax = axes[i]
    for category in [0, 1]:
        subset = df[df['Customer_Category'] == category][feature]
        label = 'Low Value' if category == 0 else 'High Value'
        ax.hist(subset, alpha=0.6, label=label, bins=20, edgecolor='black')
    ax.set_title(f'{feature} Distribution by Customer Category', fontsize=12, fontweight='bold')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
# First, encode Region for correlation
df_encoded = df.copy()
le = LabelEncoder()
df_encoded['Region_Encoded'] = le.fit_transform(df['Region'])

# Create correlation matrix
correlation_features = ['Age', 'Income', 'Spending_Score', 'Region_Encoded', 'Customer_Category']
corr_matrix = df_encoded[correlation_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, 
            linewidths=0.5, fmt='.3f', square=True)
plt.title('Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelation with Target (Customer_Category):")
print(corr_matrix['Customer_Category'].sort_values(ascending=False))

---
## 3. Data Preprocessing

In [ ]:
# Drop the unnamed index column if present
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Encode categorical variable (Region)
print("Unique Regions:", df['Region'].unique())

# One-hot encoding for Region
df_processed = pd.get_dummies(df, columns=['Region'], drop_first=False)

print("\nProcessed DataFrame columns:")
print(df_processed.columns.tolist())
df_processed.head()

In [ ]:
# Separate features and target
X = df_processed.drop('Customer_Category', axis=1)
y = df_processed['Customer_Category']

print(f"Feature Matrix Shape: {X.shape}")
print(f"Target Vector Shape: {y.shape}")
print(f"\nFeatures: {X.columns.tolist()}")

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

# Verify stratification
print(f"\nTraining set class distribution: {y_train.value_counts().to_dict()}")
print(f"Testing set class distribution: {y_test.value_counts().to_dict()}")

In [ ]:
# Feature scaling - Important for SVM!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for better visualization
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

print("Feature Scaling Applied!")
print("\nScaled Training Data Statistics:")
print(X_train_scaled_df.describe().round(2))

---
## 4. Building SVM Models

### 4.1 Linear SVM

In [ ]:
# Build Linear SVM model
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train_scaled, y_train)

# Predictions
y_pred_linear = svm_linear.predict(X_test_scaled)

# Evaluate
print("=" * 60)
print("LINEAR SVM RESULTS")
print("=" * 60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_linear):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_linear):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_linear):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_linear):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_linear, target_names=['Low Value', 'High Value']))

### 4.2 RBF (Radial Basis Function) Kernel SVM

In [ ]:
# Build RBF SVM model
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train_scaled, y_train)

# Predictions
y_pred_rbf = svm_rbf.predict(X_test_scaled)

# Evaluate
print("=" * 60)
print("RBF KERNEL SVM RESULTS")
print("=" * 60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rbf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rbf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rbf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rbf):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rbf, target_names=['Low Value', 'High Value']))

### 4.3 Polynomial Kernel SVM

In [ ]:
# Build Polynomial SVM model
svm_poly = SVC(kernel='poly', degree=3, random_state=42)
svm_poly.fit(X_train_scaled, y_train)

# Predictions
y_pred_poly = svm_poly.predict(X_test_scaled)

# Evaluate
print("=" * 60)
print("POLYNOMIAL KERNEL SVM RESULTS")
print("=" * 60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_poly):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_poly):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_poly):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_poly):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_poly, target_names=['Low Value', 'High Value']))

### 4.4 Model Comparison

In [ ]:
# Compare all models
models = {
    'Linear SVM': svm_linear,
    'RBF SVM': svm_rbf,
    'Polynomial SVM': svm_poly
}

predictions = {
    'Linear SVM': y_pred_linear,
    'RBF SVM': y_pred_rbf,
    'Polynomial SVM': y_pred_poly
}

# Create comparison table
comparison_data = []
for name, y_pred in predictions.items():
    comparison_data.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nModel Comparison:")
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart comparison
ax1 = axes[0]
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics))
width = 0.25

colors = ['#3498db', '#e74c3c', '#2ecc71']
for i, (name, row) in enumerate(comparison_df.iterrows()):
    values = [comparison_df.loc[i, m] for m in metrics]
    ax1.bar(x + i*width, values, width, label=comparison_df.loc[i, 'Model'], color=colors[i])

ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(x + width)
ax1.set_xticklabels(metrics)
ax1.legend(loc='lower right')
ax1.set_ylim([0, 1])
ax1.grid(axis='y', alpha=0.3)

# Confusion matrices
ax2 = axes[1]
best_model_name = comparison_df.loc[comparison_df['Accuracy'].idxmax(), 'Model']
best_y_pred = predictions[best_model_name]
cm = confusion_matrix(y_test, best_y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Low Value', 'High Value'])
disp.plot(ax=ax2, cmap='Blues')
ax2.set_title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 5. Hyperparameter Tuning

In [ ]:
# Grid Search for optimal hyperparameters
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print("\nBest Parameters:", grid_search.best_params_)
print(f"Best Cross-Validation Score: {grid_search.best_score_:.4f}")

In [ ]:
# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)

print("=" * 60)
print("OPTIMIZED SVM RESULTS")
print("=" * 60)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_best):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_best):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best, target_names=['Low Value', 'High Value']))

---
## 6. Analysis of Support Vectors and Feature Importance

In [ ]:
# Analyze Support Vectors (using Linear SVM for interpretability)
print("=" * 60)
print("SUPPORT VECTOR ANALYSIS")
print("=" * 60)

print(f"\nNumber of Support Vectors: {len(svm_linear.support_)}")
print(f"Support Vectors per class: {svm_linear.n_support_}")
print(f"\nTotal training samples: {len(X_train)}")
print(f"Percentage of support vectors: {len(svm_linear.support_) / len(X_train) * 100:.2f}%")

In [ ]:
# Feature Importance Analysis (for Linear SVM)
# In linear SVM, the weight vector (coef_) indicates feature importance

feature_weights = pd.DataFrame({
    'Feature': X.columns,
    'Weight': svm_linear.coef_[0],
    'Absolute_Weight': np.abs(svm_linear.coef_[0])
}).sort_values('Absolute_Weight', ascending=False)

print("\n" + "=" * 60)
print("FEATURE IMPORTANCE (Linear SVM)")
print("=" * 60)
print("\nFeature weights (sorted by absolute importance):")
print(feature_weights.to_string(index=False))

In [ ]:
# Visualize Feature Importance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart of feature weights
ax1 = axes[0]
colors = ['#e74c3c' if w < 0 else '#2ecc71' for w in feature_weights['Weight']]
bars = ax1.barh(feature_weights['Feature'], feature_weights['Weight'], color=colors, edgecolor='black')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax1.set_xlabel('Weight Coefficient')
ax1.set_title('Feature Weights in Linear SVM\n(Positive = Contributes to High Value, Negative = Contributes to Low Value)', 
              fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Absolute importance
ax2 = axes[1]
ax2.barh(feature_weights['Feature'], feature_weights['Absolute_Weight'], color='steelblue', edgecolor='black')
ax2.set_xlabel('Absolute Weight')
ax2.set_title('Feature Importance (Absolute)', fontsize=12, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Identify most important feature
most_important_feature = feature_weights.iloc[0]['Feature']
print(f"\n🔑 Most influential feature: {most_important_feature}")
print(f"   Weight: {feature_weights.iloc[0]['Weight']:.4f}")

---
## 7. Margin Analysis

In [ ]:
# Calculate margin width for Linear SVM
# Margin width = 2 / ||w|| where w is the weight vector

weight_norm = np.linalg.norm(svm_linear.coef_)
margin_width = 2 / weight_norm

print("=" * 60)
print("MARGIN ANALYSIS")
print("=" * 60)
print(f"\nWeight Vector Norm (||w||): {weight_norm:.4f}")
print(f"Margin Width (2/||w||): {margin_width:.4f}")

# Analyze margin for different C values
print("\n" + "-" * 40)
print("Margin Width for Different C Values:")
print("-" * 40)

c_values = [0.001, 0.01, 0.1, 1, 10, 100]
margin_data = []

for c in c_values:
    temp_svm = SVC(kernel='linear', C=c, random_state=42)
    temp_svm.fit(X_train_scaled, y_train)
    temp_norm = np.linalg.norm(temp_svm.coef_)
    temp_margin = 2 / temp_norm
    temp_accuracy = accuracy_score(y_test, temp_svm.predict(X_test_scaled))
    temp_n_sv = len(temp_svm.support_)
    
    margin_data.append({
        'C': c,
        'Margin_Width': temp_margin,
        'Num_Support_Vectors': temp_n_sv,
        'Test_Accuracy': temp_accuracy
    })
    
margin_df = pd.DataFrame(margin_data)
print(margin_df.to_string(index=False))

In [ ]:
# Visualize margin vs C relationship
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Margin Width vs C
ax1 = axes[0]
ax1.plot(margin_df['C'], margin_df['Margin_Width'], 'bo-', linewidth=2, markersize=8)
ax1.set_xscale('log')
ax1.set_xlabel('C (Regularization Parameter)')
ax1.set_ylabel('Margin Width')
ax1.set_title('Margin Width vs. C', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Number of Support Vectors vs C
ax2 = axes[1]
ax2.plot(margin_df['C'], margin_df['Num_Support_Vectors'], 'ro-', linewidth=2, markersize=8)
ax2.set_xscale('log')
ax2.set_xlabel('C (Regularization Parameter)')
ax2.set_ylabel('Number of Support Vectors')
ax2.set_title('Support Vectors vs. C', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Accuracy vs C
ax3 = axes[2]
ax3.plot(margin_df['C'], margin_df['Test_Accuracy'], 'go-', linewidth=2, markersize=8)
ax3.set_xscale('log')
ax3.set_xlabel('C (Regularization Parameter)')
ax3.set_ylabel('Test Accuracy')
ax3.set_title('Test Accuracy vs. C', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 8. Model Stability Analysis

In [ ]:
# Analyze model stability by retraining with different random states
print("=" * 60)
print("MODEL STABILITY ANALYSIS")
print("=" * 60)

stability_results = []
support_vector_counts = []

for seed in range(10):
    # Different train/test split
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )
    
    # Scale
    scaler_temp = StandardScaler()
    X_train_temp_scaled = scaler_temp.fit_transform(X_train_temp)
    X_test_temp_scaled = scaler_temp.transform(X_test_temp)
    
    # Train model
    svm_temp = SVC(kernel='linear', random_state=42)
    svm_temp.fit(X_train_temp_scaled, y_train_temp)
    
    # Evaluate
    y_pred_temp = svm_temp.predict(X_test_temp_scaled)
    acc = accuracy_score(y_test_temp, y_pred_temp)
    n_sv = len(svm_temp.support_)
    
    stability_results.append(acc)
    support_vector_counts.append(n_sv)

print(f"\nAccuracy across 10 different splits:")
print(f"  Mean: {np.mean(stability_results):.4f}")
print(f"  Std:  {np.std(stability_results):.4f}")
print(f"  Min:  {np.min(stability_results):.4f}")
print(f"  Max:  {np.max(stability_results):.4f}")

print(f"\nSupport Vector Count across 10 different splits:")
print(f"  Mean: {np.mean(support_vector_counts):.1f}")
print(f"  Std:  {np.std(support_vector_counts):.1f}")
print(f"  Min:  {np.min(support_vector_counts)}")
print(f"  Max:  {np.max(support_vector_counts)}")

In [ ]:
# Cross-validation for stability assessment
cv_scores = cross_val_score(SVC(kernel='linear', random_state=42), X_train_scaled, y_train, cv=10)

print("\n" + "-" * 40)
print("10-Fold Cross-Validation Results:")
print("-" * 40)
print(f"Individual Fold Scores: {cv_scores.round(4)}")
print(f"Mean CV Score: {cv_scores.mean():.4f}")
print(f"Std CV Score: {cv_scores.std():.4f}")

---
## 9. Kernel Comparison Analysis

In [ ]:
# Analyze performance across different kernels
print("=" * 60)
print("KERNEL COMPARISON ANALYSIS")
print("=" * 60)

kernels = ['linear', 'rbf', 'poly', 'sigmoid']
kernel_results = []

for kernel in kernels:
    svm_temp = SVC(kernel=kernel, random_state=42)
    
    # Cross-validation
    cv_scores = cross_val_score(svm_temp, X_train_scaled, y_train, cv=5)
    
    # Fit and predict
    svm_temp.fit(X_train_scaled, y_train)
    y_pred_temp = svm_temp.predict(X_test_scaled)
    test_acc = accuracy_score(y_test, y_pred_temp)
    
    kernel_results.append({
        'Kernel': kernel,
        'CV_Mean': cv_scores.mean(),
        'CV_Std': cv_scores.std(),
        'Test_Accuracy': test_acc,
        'Num_Support_Vectors': len(svm_temp.support_)
    })

kernel_df = pd.DataFrame(kernel_results)
print("\nKernel Performance Comparison:")
print(kernel_df.to_string(index=False))

In [ ]:
# Visualize kernel comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
ax1 = axes[0]
x = np.arange(len(kernels))
width = 0.35

bars1 = ax1.bar(x - width/2, kernel_df['CV_Mean'], width, label='CV Mean', color='steelblue', 
                yerr=kernel_df['CV_Std'], capsize=5)
bars2 = ax1.bar(x + width/2, kernel_df['Test_Accuracy'], width, label='Test Accuracy', color='coral')

ax1.set_ylabel('Accuracy')
ax1.set_title('Kernel Performance Comparison', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(kernel_df['Kernel'])
ax1.legend()
ax1.set_ylim([0.4, 0.7])
ax1.grid(axis='y', alpha=0.3)

# Support vectors comparison
ax2 = axes[1]
ax2.bar(kernel_df['Kernel'], kernel_df['Num_Support_Vectors'], color='purple', edgecolor='black')
ax2.set_ylabel('Number of Support Vectors')
ax2.set_title('Support Vectors by Kernel', fontsize=12, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 10. Answering the Questions

### Question 1: How do support vectors impact the decision boundary in SVM? Which feature has the strongest influence on customer segmentation?

In [ ]:
print("=" * 80)
print("QUESTION 1: Support Vectors and Feature Importance")
print("=" * 80)

print("""
📌 HOW SUPPORT VECTORS IMPACT THE DECISION BOUNDARY:

Support vectors are the critical data points that lie closest to the decision boundary
(hyperplane) in SVM. They have a fundamental impact on the decision boundary in the
following ways:

1. **Definition of the Hyperplane**: The decision boundary is determined SOLELY by the
   support vectors. All other data points have no influence on the position of the
   hyperplane.

2. **Margin Determination**: Support vectors define the margin - the distance between
   the decision boundary and the nearest data points. The SVM algorithm maximizes this
   margin to improve generalization.

3. **Robustness**: Because only support vectors matter, the model is robust to changes
   in non-support vector points. This is a key advantage of SVMs.

4. **Computational Efficiency**: During prediction, only support vectors are used,
   making the model efficient even with large datasets.
""")

print(f"\n📊 In our customer segmentation model:")
print(f"   - Total training samples: {len(X_train)}")
print(f"   - Number of support vectors: {len(svm_linear.support_)}")
print(f"   - Percentage: {len(svm_linear.support_) / len(X_train) * 100:.2f}%")
print(f"   - Support vectors per class: {svm_linear.n_support_}")

print("\n" + "="*60)
print("\n📌 FEATURE WITH STRONGEST INFLUENCE:\n")
print(feature_weights.to_string(index=False))

print(f"\n🔑 ANSWER: The feature with the STRONGEST influence on customer segmentation is:")
print(f"   Feature: {feature_weights.iloc[0]['Feature']}")
print(f"   Weight: {feature_weights.iloc[0]['Weight']:.4f}")
print(f"   Absolute Importance: {feature_weights.iloc[0]['Absolute_Weight']:.4f}")

### Question 2: Implications of Margin Width, Non-linear Kernels, and Support Vector Stability

In [ ]:
print("=" * 80)
print("QUESTION 2: Margin Width, Non-linear Kernels, and Model Stability")
print("=" * 80)

print("""
📌 IF THE MARGIN WIDTH IS LARGE, WHAT DOES IT IMPLY ABOUT CLASSIFICATION CONFIDENCE?

A LARGE MARGIN implies:

1. **Higher Classification Confidence**: When the margin is wide, data points have a
   larger "buffer zone" between classes. Points that fall well within their respective
   sides can be classified with higher confidence.

2. **Better Generalization**: A wider margin typically indicates the model will perform
   well on unseen data, as it's not overfitting to the training examples.

3. **Robustness to Noise**: The model can tolerate small perturbations in the input data
   without changing its predictions.

4. **Lower Variance**: The model is less sensitive to the specific training examples used.

TRADE-OFF: Very large margins (low C) might lead to underfitting, while very small
margins (high C) might lead to overfitting.
""")

print("\n📊 Evidence from our analysis:")
print(margin_df.to_string(index=False))

print("\n" + "="*60)

print("""
📌 IF THE DATA REQUIRES A NON-LINEAR KERNEL, WHAT MIGHT THAT SUGGEST ABOUT THE DATASET?

The need for a non-linear kernel suggests:

1. **Non-Linear Decision Boundary**: The classes cannot be separated by a straight line
   (hyperplane) in the original feature space. The relationship between features and
   the target is complex.

2. **Complex Feature Interactions**: There may be important interactions between features
   that a linear model cannot capture.

3. **Higher-Dimensional Separability**: The kernel trick maps data to a higher-dimensional
   space where linear separation becomes possible.

4. **Data Distribution**: The data might have clusters, spirals, or other non-linear
   patterns that require curved decision boundaries.
""")

print("\n📊 Evidence from our kernel comparison:")
print(kernel_df.to_string(index=False))

linear_acc = kernel_df[kernel_df['Kernel'] == 'linear']['Test_Accuracy'].values[0]
rbf_acc = kernel_df[kernel_df['Kernel'] == 'rbf']['Test_Accuracy'].values[0]

if rbf_acc > linear_acc:
    print(f"\n🔍 In our dataset: RBF kernel ({rbf_acc:.4f}) outperforms Linear ({linear_acc:.4f}),")
    print("   suggesting the data has some non-linear patterns.")
else:
    print(f"\n🔍 In our dataset: Linear kernel ({linear_acc:.4f}) performs comparably to RBF ({rbf_acc:.4f}),")
    print("   suggesting the data is relatively linearly separable.")

print("\n" + "="*60)

print("""
📌 IF SUPPORT VECTORS CHANGE SIGNIFICANTLY WHEN RETRAINED, WHAT DOES THAT IMPLY ABOUT
   MODEL STABILITY?

Significant changes in support vectors indicate:

1. **Low Model Stability**: The model is sensitive to the specific training examples,
   which can lead to inconsistent predictions on new data.

2. **Overlapping Classes**: The data may have significant overlap between classes,
   making the boundary definition ambiguous.

3. **Insufficient or Noisy Data**: There might not be enough data, or the data may
   contain noise that affects which points become support vectors.

4. **High Variance**: Small changes in the training data lead to different models,
   indicating high variance.

5. **Need for Regularization**: The C parameter might need adjustment - a smaller C
   allows more flexibility and might lead to more stable support vectors.
""")

print("\n📊 Evidence from our stability analysis:")
print(f"   Support Vector Count across splits:")
print(f"   - Mean: {np.mean(support_vector_counts):.1f}")
print(f"   - Std:  {np.std(support_vector_counts):.1f}")
print(f"   - Range: {np.min(support_vector_counts)} to {np.max(support_vector_counts)}")
print(f"\n   Accuracy across splits:")
print(f"   - Mean: {np.mean(stability_results):.4f}")
print(f"   - Std:  {np.std(stability_results):.4f}")

sv_variation = np.std(support_vector_counts) / np.mean(support_vector_counts) * 100
if sv_variation > 10:
    stability_assessment = "potentially unstable (high variation in support vectors)"
else:
    stability_assessment = "relatively stable (low variation in support vectors)"

print(f"\n🔍 Our model is {stability_assessment}")
print(f"   Coefficient of variation for support vectors: {sv_variation:.2f}%")

---
## 11. Summary and Conclusions

In [ ]:
print("=" * 80)
print("SUMMARY AND CONCLUSIONS")
print("=" * 80)

print("""
📋 KEY FINDINGS:

1. MODEL PERFORMANCE:
""")
print(f"   - Best performing kernel: {comparison_df.loc[comparison_df['Accuracy'].idxmax(), 'Model']}")
print(f"   - Best test accuracy: {comparison_df['Accuracy'].max():.4f}")
print(f"   - After hyperparameter tuning: {accuracy_score(y_test, y_pred_best):.4f}")

print(f"""
2. FEATURE IMPORTANCE:
   - Most influential feature: {feature_weights.iloc[0]['Feature']}
   - The numerical features (Age, Income, Spending_Score) generally show
     stronger influence than the regional categorical variables.

3. SUPPORT VECTOR ANALYSIS:
   - Number of support vectors: {len(svm_linear.support_)} out of {len(X_train)} training samples
   - This represents {len(svm_linear.support_) / len(X_train) * 100:.2f}% of the training data
   - A moderate percentage indicates good margin definition

4. MODEL STABILITY:
   - Cross-validation mean accuracy: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})
   - The model shows {'good' if cv_scores.std() < 0.05 else 'moderate'} stability

5. DATA CHARACTERISTICS:
   - The similar performance of linear and non-linear kernels suggests
     the data is relatively linearly separable
   - The margin analysis shows typical trade-off between margin width
     and number of support vectors

📌 RECOMMENDATIONS:
   - The optimized model with parameters {grid_search.best_params_} provides
     the best balance of accuracy and generalization
   - Consider feature engineering to improve prediction accuracy
   - The model can be deployed for customer segmentation with reasonable confidence
""")

In [ ]:
# Final model summary
print("\n" + "=" * 80)
print("FINAL MODEL CONFIGURATION")
print("=" * 80)
print(f"\nBest Model Parameters: {grid_search.best_params_}")
print(f"Test Set Performance:")
print(f"  - Accuracy:  {accuracy_score(y_test, y_pred_best):.4f}")
print(f"  - Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"  - Recall:    {recall_score(y_test, y_pred_best):.4f}")
print(f"  - F1-Score:  {f1_score(y_test, y_pred_best):.4f}")